# Coloring B&W images 

#### Using OpenCv library for performing computer vision related operations

In [1]:
import tensorflow as tf
from tensorflow.keras import backend
backend.clear_session()
import numpy as np
import os
import random
from matplotlib import pyplot as plt

from tensorflow.keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from tensorflow.keras.layers import Activation, Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

#### Converting RGB image to LAB space image

The CIELAB color space, also referred to as L*a*b* , is a color space defined by the International Commission on Illumination.t expresses color as three values: L* for perceptual lightness, and a* and b* for the four unique colors of human vision: red, green, blue, and yellow. CIELAB was intended as a perceptually uniform space, where a given numerical change corresponds to a similar perceived change in color. While the LAB space is not truly perceptually uniform, it nevertheless is useful in industry for detecting small differences in color.

In [2]:
import cv2
#load RGB image
original_img = cv2.imread('girl.jpg')
print(original_img.shape)

#convert GRAY to LAB image
lab_image = cv2.cvtColor(original_img, cv2.COLOR_RGB2LAB)
lab_image = np.array(lab_image, dtype=float)

X = lab_image[:,:,0]*(1.0/255)
Y = lab_image[:,:,1:]*(1.0/255)

Y = Y/128
X = X.reshape(1, lab_image.shape[0], lab_image.shape[1], 1)
Y = Y.reshape(1, lab_image.shape[0], lab_image.shape[1], 2)

(340, 638, 3)


In [3]:
# Building the neural network   
model = Sequential()
model.add(InputLayer(input_shape=(None, None,1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [4]:
# Finish model
model.compile(optimizer='adam',loss='mse')

#### Increase epochs for Improving efficiency of the model

In [5]:
model.fit(x=X, 
    y=Y,
    batch_size=1,
    epochs=2000)

Epoch 1/2000
1/1 [==============================] - 1s 1s/step - loss: 4.5359e-05
Epoch 2/2000
1/1 [==============================] - 0s 267ms/step - loss: 6.6049e-05
Epoch 3/2000
1/1 [==============================] - 0s 267ms/step - loss: 2.0802e-05
Epoch 4/2000
1/1 [==============================] - 0s 260ms/step - loss: 4.2719e-06
Epoch 5/2000
1/1 [==============================] - 0s 273ms/step - loss: 8.3615e-06
Epoch 6/2000
1/1 [==============================] - 0s 264ms/step - loss: 8.1887e-06
Epoch 7/2000
1/1 [==============================] - 0s 252ms/step - loss: 3.8445e-06
Epoch 8/2000
1/1 [==============================] - 0s 251ms/step - loss: 1.3210e-06
Epoch 9/2000
1/1 [==============================] - 0s 262ms/step - loss: 1.0025e-06
Epoch 10/2000
1/1 [==============================] - 0s 288ms/step - loss: 1.4016e-06
Epoch 11/2000
1/1 [==============================] - 0s 287ms/step - loss: 1.6568e-06
Epoch 12/2000
1/1 [==============================] - 0s 326ms/step

1/1 [==============================] - 0s 250ms/step - loss: 1.5830e-07
Epoch 97/2000
1/1 [==============================] - 0s 219ms/step - loss: 1.5807e-07
Epoch 98/2000
1/1 [==============================] - 0s 250ms/step - loss: 1.5782e-07
Epoch 99/2000
1/1 [==============================] - 0s 250ms/step - loss: 1.5752e-07
Epoch 100/2000
1/1 [==============================] - 0s 250ms/step - loss: 1.5717e-07
Epoch 101/2000
1/1 [==============================] - 0s 250ms/step - loss: 1.5681e-07
Epoch 102/2000
1/1 [==============================] - 0s 234ms/step - loss: 1.5644e-07
Epoch 103/2000
1/1 [==============================] - 0s 250ms/step - loss: 1.5609e-07
Epoch 104/2000
1/1 [==============================] - 0s 250ms/step - loss: 1.5578e-07
Epoch 105/2000
1/1 [==============================] - 0s 250ms/step - loss: 1.5555e-07
Epoch 106/2000
1/1 [==============================] - 0s 234ms/step - loss: 1.5544e-07
Epoch 107/2000
1/1 [==============================] - 0s 266m

1/1 [==============================] - 0s 250ms/step - loss: 1.2892e-07
Epoch 191/2000
1/1 [==============================] - 0s 266ms/step - loss: 1.2866e-07
Epoch 192/2000
1/1 [==============================] - 0s 281ms/step - loss: 1.2840e-07
Epoch 193/2000
1/1 [==============================] - 0s 281ms/step - loss: 1.2816e-07
Epoch 194/2000
1/1 [==============================] - 0s 297ms/step - loss: 1.2793e-07
Epoch 195/2000
1/1 [==============================] - 0s 281ms/step - loss: 1.2770e-07
Epoch 196/2000
1/1 [==============================] - 0s 281ms/step - loss: 1.2747e-07
Epoch 197/2000
1/1 [==============================] - 0s 281ms/step - loss: 1.2724e-07
Epoch 198/2000
1/1 [==============================] - 0s 281ms/step - loss: 1.2700e-07
Epoch 199/2000
1/1 [==============================] - 0s 266ms/step - loss: 1.2676e-07
Epoch 200/2000
1/1 [==============================] - 0s 266ms/step - loss: 1.2652e-07
Epoch 201/2000
1/1 [==============================] - 0s 2

1/1 [==============================] - 0s 250ms/step - loss: 1.1294e-07
Epoch 285/2000
1/1 [==============================] - 0s 250ms/step - loss: 1.1284e-07
Epoch 286/2000
1/1 [==============================] - 0s 250ms/step - loss: 1.1275e-07
Epoch 287/2000
1/1 [==============================] - 0s 250ms/step - loss: 1.1266e-07
Epoch 288/2000
1/1 [==============================] - 0s 234ms/step - loss: 1.1256e-07
Epoch 289/2000
1/1 [==============================] - 0s 250ms/step - loss: 1.1247e-07
Epoch 290/2000
1/1 [==============================] - 0s 234ms/step - loss: 1.1238e-07
Epoch 291/2000
1/1 [==============================] - 0s 250ms/step - loss: 1.1229e-07
Epoch 292/2000
1/1 [==============================] - 0s 234ms/step - loss: 1.1220e-07
Epoch 293/2000
1/1 [==============================] - 0s 250ms/step - loss: 1.1211e-07
Epoch 294/2000
1/1 [==============================] - 0s 234ms/step - loss: 1.1202e-07
Epoch 295/2000
1/1 [==============================] - 0s 2

1/1 [==============================] - 0s 234ms/step - loss: 1.0656e-07
Epoch 379/2000
1/1 [==============================] - 0s 281ms/step - loss: 1.0650e-07
Epoch 380/2000
1/1 [==============================] - 0s 281ms/step - loss: 1.0645e-07
Epoch 381/2000
1/1 [==============================] - 0s 281ms/step - loss: 1.0639e-07
Epoch 382/2000
1/1 [==============================] - 0s 281ms/step - loss: 1.0633e-07
Epoch 383/2000
1/1 [==============================] - 0s 281ms/step - loss: 1.0627e-07
Epoch 384/2000
1/1 [==============================] - 0s 281ms/step - loss: 1.0622e-07
Epoch 385/2000
1/1 [==============================] - 0s 281ms/step - loss: 1.0616e-07
Epoch 386/2000
1/1 [==============================] - 0s 266ms/step - loss: 1.0610e-07
Epoch 387/2000
1/1 [==============================] - 0s 250ms/step - loss: 1.0605e-07
Epoch 388/2000
1/1 [==============================] - 0s 281ms/step - loss: 1.0599e-07
Epoch 389/2000
1/1 [==============================] - 0s 2

...

1/1 [==============================] - 0s 258ms/step - loss: 5.9488e-08
Epoch 1595/2000
1/1 [==============================] - 0s 265ms/step - loss: 5.7156e-08
Epoch 1596/2000
1/1 [==============================] - 0s 266ms/step - loss: 5.6158e-08
Epoch 1597/2000
1/1 [==============================] - 0s 255ms/step - loss: 5.6468e-08
Epoch 1598/2000
1/1 [==============================] - 0s 269ms/step - loss: 5.7562e-08
Epoch 1599/2000
1/1 [==============================] - 0s 259ms/step - loss: 5.9435e-08
Epoch 1600/2000
1/1 [==============================] - 0s 260ms/step - loss: 6.0111e-08
Epoch 1601/2000
1/1 [==============================] - 0s 249ms/step - loss: 6.1804e-08
Epoch 1602/2000
1/1 [==============================] - 0s 253ms/step - loss: 5.9975e-08
Epoch 1603/2000
1/1 [==============================] - 0s 256ms/step - loss: 5.9211e-08
Epoch 1604/2000
1/1 [==============================] - 0s 257ms/step - loss: 5.7408e-08
Epoch 1605/2000
1/1 [===========================

1/1 [==============================] - 0s 279ms/step - loss: 5.5738e-08
Epoch 1688/2000
1/1 [==============================] - 0s 273ms/step - loss: 5.7418e-08
Epoch 1689/2000
1/1 [==============================] - 0s 328ms/step - loss: 5.8940e-08
Epoch 1690/2000
1/1 [==============================] - 0s 251ms/step - loss: 6.1663e-08
Epoch 1691/2000
1/1 [==============================] - 0s 283ms/step - loss: 5.9529e-08
Epoch 1692/2000
1/1 [==============================] - 0s 250ms/step - loss: 5.8522e-08
Epoch 1693/2000
1/1 [==============================] - 0s 250ms/step - loss: 5.6191e-08
Epoch 1694/2000
1/1 [==============================] - 0s 268ms/step - loss: 5.5061e-08
Epoch 1695/2000
1/1 [==============================] - 0s 258ms/step - loss: 5.4697e-08
Epoch 1696/2000
1/1 [==============================] - 0s 265ms/step - loss: 5.5091e-08
Epoch 1697/2000
1/1 [==============================] - 0s 250ms/step - loss: 5.5999e-08
Epoch 1698/2000
1/1 [===========================

1/1 [==============================] - 0s 285ms/step - loss: 5.8414e-08
Epoch 1781/2000
1/1 [==============================] - 0s 269ms/step - loss: 6.4586e-08
Epoch 1782/2000
1/1 [==============================] - 0s 248ms/step - loss: 6.3607e-08
Epoch 1783/2000
1/1 [==============================] - 0s 266ms/step - loss: 6.3336e-08
Epoch 1784/2000
1/1 [==============================] - 0s 257ms/step - loss: 5.4706e-08
Epoch 1785/2000
1/1 [==============================] - 0s 250ms/step - loss: 5.6490e-08
Epoch 1786/2000
1/1 [==============================] - 0s 252ms/step - loss: 6.5867e-08
Epoch 1787/2000
1/1 [==============================] - 0s 266ms/step - loss: 6.0390e-08
Epoch 1788/2000
1/1 [==============================] - 0s 249ms/step - loss: 5.6040e-08
Epoch 1789/2000
1/1 [==============================] - 0s 258ms/step - loss: 5.4743e-08
Epoch 1790/2000
1/1 [==============================] - 0s 252ms/step - loss: 5.7692e-08
Epoch 1791/2000
1/1 [===========================

1/1 [==============================] - 0s 259ms/step - loss: 5.2614e-08
Epoch 1874/2000
1/1 [==============================] - 0s 270ms/step - loss: 5.2458e-08
Epoch 1875/2000
1/1 [==============================] - 0s 264ms/step - loss: 5.2908e-08
Epoch 1876/2000
1/1 [==============================] - 0s 258ms/step - loss: 5.3753e-08
Epoch 1877/2000
1/1 [==============================] - 0s 249ms/step - loss: 5.4196e-08
Epoch 1878/2000
1/1 [==============================] - 0s 243ms/step - loss: 5.5076e-08
Epoch 1879/2000
1/1 [==============================] - 0s 252ms/step - loss: 5.5282e-08
Epoch 1880/2000
1/1 [==============================] - 0s 265ms/step - loss: 5.6470e-08
Epoch 1881/2000
1/1 [==============================] - 0s 257ms/step - loss: 5.5995e-08
Epoch 1882/2000
1/1 [==============================] - 0s 252ms/step - loss: 5.6243e-08
Epoch 1883/2000
1/1 [==============================] - 0s 266ms/step - loss: 5.4368e-08
Epoch 1884/2000
1/1 [===========================

1/1 [==============================] - 0s 266ms/step - loss: 5.6066e-08
Epoch 1967/2000
1/1 [==============================] - 0s 299ms/step - loss: 5.7240e-08
Epoch 1968/2000
1/1 [==============================] - 0s 291ms/step - loss: 5.8162e-08
Epoch 1969/2000
1/1 [==============================] - 0s 325ms/step - loss: 5.3983e-08
Epoch 1970/2000
1/1 [==============================] - 0s 291ms/step - loss: 5.1954e-08
Epoch 1971/2000
1/1 [==============================] - 0s 286ms/step - loss: 5.2070e-08
Epoch 1972/2000
1/1 [==============================] - 0s 316ms/step - loss: 5.3443e-08
Epoch 1973/2000
1/1 [==============================] - 0s 273ms/step - loss: 5.5625e-08
Epoch 1974/2000
1/1 [==============================] - 0s 279ms/step - loss: 5.5595e-08
Epoch 1975/2000
1/1 [==============================] - 0s 285ms/step - loss: 5.6299e-08
Epoch 1976/2000
1/1 [==============================] - 0s 291ms/step - loss: 5.4376e-08
Epoch 1977/2000
1/1 [===========================

In [6]:
print(model.evaluate(X, Y, batch_size=1))
output = model.predict(X)
output *= 128*255
X*=255
# Output colorizations
cur = np.zeros((lab_image.shape[0],lab_image.shape[1], 3), dtype=np.uint8)
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]

output_image = cv2.cvtColor(cur, cv2.COLOR_LAB2RGB)
Input_image = cv2.cvtColor(original_img, cv2.COLOR_RGB2GRAY)

1/1 [==============================] - 1s 540ms/step - loss: 5.4680e-08
5.4679738070717576e-08


In [7]:
cv2.imshow('Ouput', output_image)
cv2.imshow('Input', Input_image)
cv2.waitKey(0)
cv2.destroyAllWindows()